<a href="https://colab.research.google.com/github/veraDanilova/COV19_user-topic_networks/blob/main/COVID_19_topic_user_relations_dynamics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DASHBOARD APP: Weekly changes of User Attention to COVID-related topics (March 22 - June 1, 2020)

In [ ]:
"""
Additional data:
- plots: https://bit.ly/3z4dMqG
- ready graphs: https://bit.ly/3sGSLjE
"""

# install modules
!pip install pyLDAvis
!pip install jupyter_dash
!pip install dash_bootstrap_components
!pip install -U pandas-profiling
!pip install dash-table

# load Datasets from github
!git clone https://github.com/veraDanilova/COV19_user-topic_networks.git

# COPY_FILE = True
! unzip /content/COV19_user-topic_networks/Proj_Data/COR_LJ_resized.csv.zip -d /content/COV19_user-topic_networks/Proj_Data/
! unzip /content/COV19_user-topic_networks/Proj_Data/COR_TWI_resized.csv.zip -d /content/COV19_user-topic_networks/Proj_Data/

# %%writefile COV19_user-topic_networks.py
import sys
# sys.path.append('/content/drive/MyDrive/COV19_user-topic_networks/')
sys.path.append('/content/COV19_user-topic_networks/Proj_Data/')

"""
BIDIRECTED GRAPH OF USER-TOPIC RELATIONS

"""
# IMPORT MODULES
# general
import re
import string
import gzip
import os
import itertools
import sklearn.preprocessing

# data
import pandas as pd
import json
import gensim
import gensim.corpora as corpora


# LDA
from gensim.models import CoherenceModel
from gensim.models.wrappers import LdaMallet
import pyLDAvis

# Plotting tools
import matplotlib.pyplot as plt
# %matplotlib inline
import networkx as nx
import plotly.io as pio
pio.renderers.default='notebook'
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot, plot
init_notebook_mode(connected=True)

from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate


from BuildGraphT2 import GraphType2D

# root_folder = '/content/drive/MyDrive/COV19_user-topic_networks/'
root_folder = '/content/COV19_user-topic_networks/Proj_Data/'

# G = GraphType2D("Twi", "w2", 0, root_folder)
# G.plotgraphType2()

# proxy = JupyterDash.infer_jupyter_proxy_config()


# external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# app = JupyterDash(__name__, external_stylesheets=external_stylesheets)


app = JupyterDash(__name__,external_stylesheets=[dbc.themes.COSMO])

wnums = ['w1', 'w2', 'w3', 'w4', 'w5', 'w6', 'w7', 'w8', 'w9', 'w10']
week_dates = [("2020-03-22 - 2020-03-29"), 
        ("2020-03-29 - 2020-04-05"),
        ("2020-04-05 - 2020-04-12"),
        ("2020-04-12 - 2020-04-19"),
        ("2020-04-19 - 2020-04-26"),
        ("2020-04-26 - 2020-05-03"),
        ("2020-05-03 - 2020-05-10"),
        ("2020-05-10 - 2020-05-17"),
        ("2020-05-17 - 2020-05-24"),
        ("2020-05-24 - 2020-06-01")]


# cluster_input = html.Div(
#     [
# #         dbc.Label("Cluster: "),
#         dbc.RadioItems(
#             options=[{'label': t, 'value': i} for i,t in zip([0, 1],['uniform activity','one-week users'])],
#             value=0,
#             id="cluster-filter",
#             className="mr-1",
# #             inline=True
#         ),
#     ], 
#     style = {"margin-right": "2rem"})

# sm_input = html.Div(
#     [
# #         dbc.Label("Social media"),
#         dbc.RadioItems(
#             id='net-filter',
#             options=[{'label': i, 'value': i} for i in ['LJ', 'Twi']],
#             value='LJ',
#             className="mr-3",
# )
#     ], 
#     style = {"margin-left": "3rem", "margin-right": "2rem"})


# inline_inputs = dbc.Form(
#     dbc.Row(
#         [sm_input, cluster_input]
#         )
# )

inline_inputs = dbc.Form(
    dbc.Row(
        [
         dbc.RadioItems(
            options=[{'label': t, 'value': i} for i,t in zip([0, 1],['uniform activity','one-week users'])],
            value=0,
            id="cluster-filter",
            className="mr-1",
            ),
         dbc.RadioItems(
             id='net-filter',
             options=[{'label': i, 'value': i} for i in ['LJ', 'Twi']],
             value='LJ',
             className="mr-3"
             )
         ]
)
)

app.layout = html.Div(
    [
        html.H1("Weekly changes of User Attention to COVID-related topics (March 22 - June 1, 2020)", style = {'color': 'navy'}),
        html.Div([inline_inputs,
                  html.Div([dcc.Dropdown(
                      id='week-filter',
                      options=[{'label': i +' : '+ wd, 'value': i} for i,wd in zip(wnums, week_dates)],
                      value='w1')], 
                      style = {'width': '30%'})], 
                            style={'display': 'flex'}),
        # html.Div(id = 'dd-output-container')
        html.Div([dcc.Graph(id = 'dd-output-container')], style = {"margin-left": "10rem"})
    ])
################## 

@app.callback(
    dash.dependencies.Output('dd-output-container', 'figure'),
    # dash.dependencies.Output('dd-output-container', component_property='div'),
  
    [dash.dependencies.Input('week-filter', 'value'),
    dash.dependencies.Input('cluster-filter', 'value'),
    dash.dependencies.Input('net-filter', 'value')
    ]
)

def update_output(weekfilter, clusterfilter, netfilter):
    
    net = netfilter
    wnum = weekfilter
    cl_id = clusterfilter
    rules = [net == "Twi", wnum == 'w2', cl_id == 1]
    if all(rules):
      fig = go.Figure()
      # fig = {}
      fig.update_layout(
          xaxis =  { "visible": False },
          yaxis = { "visible": False },
          annotations = [{"text": "There is no cluster of temporal activity (one-week peak) in week 2",
                          "xref": "paper",
                          "yref": "paper",
                          "showarrow": False,
                          "font": {"size": 28}}])

    else:
      G = GraphType2D(net, wnum, cl_id, root_folder)
      fig, stats = G.plotgraphType2()
    return fig

if __name__ == '__main__':
    app.run_server(mode = "external", debug=True)

fatal: destination path 'COV19_user-topic_networks' already exists and is not an empty directory.
Archive:  /content/COV19_user-topic_networks/Proj_Data/COR_LJ_resized.csv.zip
replace /content/COV19_user-topic_networks/Proj_Data/COR_LJ_resized.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# app._terminate_server_for_port("localhost", 8050)